In [3]:
import pandas as pd

# Sample media production service quote
data = {
    "STT": [1, 2, 3, 4, 5],
    "Tên Dịch Vụ": [
        "Quay phim sự kiện",
        "Dịch vụ dựng phim",
        "Chỉnh màu (Color Grading)",
        "Thiết kế Motion Graphics",
        "Lồng tiếng & Hậu kỳ âm thanh"
    ],
    "Mô Tả Ngắn": [
        "Quay phim Full HD, 2 cameramen, 4 giờ",
        "Dựng, cắt ghép, hiệu ứng cơ bản, 5-7 phút video",
        "Tối ưu ánh sáng, tone màu cinematic cho video",
        "Thiết kế intro/outro, đồ họa chuyển động ngắn",
        "Chỉnh âm, mix nhạc, lồng tiếng cho video"
    ],
    "Đơn Vị Tính": ["Gói", "Gói", "Gói", "Gói", "Gói"],
    "Số Lượng": [1, 1, 1, 1, 1],
    "Đơn Giá (₫)": [5000000, 3000000, 2000000, 1500000, 1000000],
}

df = pd.DataFrame(data)

# Save to CSV
file_path = "./service_quote_media.csv"
df.to_csv(file_path, index=False)


In [4]:
import csv
docs = []
with open("service_quote_media.csv", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    for row in reader:
        # Gom thông tin từng dịch vụ thành tập text
        text = (f"STT: {row['STT']}\n"
                f"Dịch vụ: {row['Tên Dịch Vụ']}\n"
                f"Mô tả: {row['Mô Tả Ngắn']}\n"
                f"Số lượng: {row['Số Lượng']}\n"
                f"Đơn giá: {row['Đơn Giá (₫)']}\n")
        docs.append(text)

In [5]:
# LLM
from langchain_ollama import OllamaLLM
from langchain.schema import HumanMessage


llm = OllamaLLM(
    model="llama3.2", 
    base_url="http://localhost:11434",
    temperature=0
)

# Send a message to the chat model
response = llm.invoke([HumanMessage(content="Hello! Can you explain what AI agents are?")])

# Print the response
print(response)

Hello! I'd be happy to explain what AI agents are.

An AI agent is a software program that can perceive its environment, take actions, and learn from experience to achieve a specific goal. In other words, an AI agent is a computer system that can think and act like a human being, but with the help of artificial intelligence (AI) technology.

AI agents typically consist of three main components:

1. **Perception**: This refers to how the agent senses its environment, such as through sensors or data feeds.
2. **Action**: This is what the agent does in response to its perception, such as moving a robot arm or sending an email.
3. **Learning**: This is how the agent improves its performance over time, often by analyzing its experiences and adjusting its behavior accordingly.

AI agents can be found in various applications, including:

* Robotics: Autonomous vehicles, drones, and robots that can navigate and interact with their environment.
* Virtual assistants: Siri, Alexa, and Google Assi

In [6]:
# Embedding
from langchain_ollama.embeddings import OllamaEmbeddings


embeddings = OllamaEmbeddings(
    model="nomic-embed-text", 
    base_url="http://localhost:11434"  # Adjust the base URL if needed
)

In [7]:
# Persist the embeddings

from langchain_chroma import Chroma

# vectordb = Chroma.from_texts(texts=docs, embedding=embeddings, persist_directory="./chroma_media")
# vectordb.persist()

vector_store = Chroma(
    embedding_function=embeddings,
    persist_directory="./chroma_media",
    collection_name="media_service",
)


vectordb = Chroma.from_texts(texts=docs, embedding=embeddings, persist_directory="./chroma_media")

In [ ]:
from langchain_core.prompts import PromptTemplate

# template = """Sử dụng nội dung sau để trả lời câu hỏi:
# nếu bạn không biết câu trả lời, hãy nói rằng bạn không biết, đừng cố gắng đoán hoặc bịa ra câu trả lời, và nếu không biết thì hãy trả lời: "Hãy để mình hỏi lại anh Quynh nhé!".

# {context}

# Câu hỏi: {question}

# Câu trả lời:"""
# custom_rag_prompt = PromptTemplate.from_template(template)
# example_messages = custom_rag_prompt.invoke(
#     {"context": "(bối cảnh ở đây)", "question": "(câu hỏi ở đây)"}
# ).to_messages()

template = """
Bạn là Piccolo Media Bot – trợ lý bán hàng chuyên về tư vấn dịch vụ media production cho Piccolo Media.
Hãy trả lời dựa trên thông tin báo giá được truy xuất dưới đây, đảm bảo:
- Phong cách trả lời tự nhiên, giống như một người thật.
- Tùy theo cách xưng hô của khách hàng mà bạn sẽ xưng hô cho phù hợp.
- Dùng tiếng Việt.
- Nếu không có đủ dữ liệu, đáp: "Xin lỗi, mình chưa rõ lắm." hoặc "Hãy để mình hỏi lại anh Quynh nhé!".
- Không đoán hoặc bịa ra câu trả lời.

Thông tin báo giá (context):
{context}

Khách hỏi: {question}
Bot trả lời:
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["context", "question"]
)

In [10]:
from langchain.chains import RetrievalQA

retriever = vectordb.as_retriever(search_kwargs={"k": 5})
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt},
)

# 5. Demo thử
query = "Làm clip 5 phút bao nhiêu tiền?"
answer = qa_chain.invoke(query)
print(answer)

{'query': 'Làm clip 5 phút bao nhiêu tiền?', 'result': 'Bạn có muốn làm clip 5 phút không? Theo thông tin báo giá mà anh Quynh đã cung cấp, dịch vụ dựng phim với thời lượng 5-7 phút và số lượng 1 sẽ có giá khoảng 3.000.000 đồng.'}
